# Reinforcement Learning Lesson #1 Q-Learning


## Markov Decision Process Components 

1. **Agent**: Part of the system that is actually going to take actions and actively effect the environment.
1. **Environment**: Part of the system that is responsibel for the effect of the actions that an agent takes
1. **Action**: The action an agent takes
1. **Reward**: The reqard the agent recives from the environment based on the action the agent takes
1. **State**: Contains all the information from the environment that is needed to take an action (current action only depends on current state)

### Block Diagram

![Markov Decision Process](https://miro.medium.com/max/1000/1*4EYA7briZGjnnhqct-tgXw.png "Markov Decision Process")


## Expected Return

The total reward expected from a state at time t is given by:

$$ G_t = R_{t+1} + R_{t+2} + \cdots + R_T $$

where $T$ is the final time step.


## Discounted Expected Return

Discount the effect of future rewards so as to indicate that immediate rewards have more importance as compared to rewards obtained far into the future.

$$ 
\begin{split}
    G_t & = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \cdots + \gamma ^ n R_T \\
    & = \sum_{k=0}^{\infty} \gamma^{k+1} R_{t+k+1} 
\end{split}
$$

where $\gamma$ is the discount such that $0 \leq \gamma \leq 1$

$$ \gamma = 0 \Rightarrow Value only immediate reward only $$

Recurrsive equation:

$$ G_t = R_{t+1} + \gamma G_{t+1} $$


## Policy Function

Probability distribution of an agent taking an action $a$ given a state $s$ is called a policy.

This means that at time $t$, under the policy $\pi$, the probability of the agent taking the action $a$ given that it is in state $s$ is equal to $\pi(a|s)$. 

The goal is to find the optimal policy, more on this latter.


## State-Value Function

This tells us how good a state is for an agent while the agent is following a given policy.

$$ v_\pi(s) = E \: [G_t | S_t = s] $$

## Action-Value Function

This tells us how good a given action a is from $ a $ state $ s $ while the agent is following policy $ \pi $

$$ q_\pi(s,a) = E \: [G_t | S_t = s, A_t = a] $$

## Optiomal Policy

A policy such that the reward obtained by following this policy is equal to or greater that the reward that can be achieved by following any other policy.

## Optimal State-Value Function

$$ v_*(s) = max \; v_\pi(s) \; \forall \; \pi $$


## Optimal Action-Value Function

$$ q_*(s, a) = max \; q_\pi(s, a) \; \forall \; \pi $$

## Bellmam Optimality  for $q_*$

$$ q_*(s, a) = E \: [R_{t+1} + \gamma max q_*(s', a')] $$


## Update Rule

$$ q_*(s,a) = (1-\alpha) q_*(s,a) + \alpha (R_{t+1} + \gamma max q_*(s', a')) $$

where $\alpha$ is the learning rate

In [1]:
# Imports
import numpy as np
import gym
import time
import random
from IPython.display import clear_output

# Set Random Seed to get consistant results on every run
random.seed(0)

In [2]:
# Get the environment
env = gym.make('FrozenLake-v0')

In [3]:
# Get total number of possible states and actions
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

In [4]:
# Q-Function (initialize with zeros)
q_table = np.zeros((state_space_size, action_space_size))

In [5]:
# Set Hyperparameter Values
num_episodes = 10000
max_steps_per_episode = 100

learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.001
exploration_decay_rate = 0.001

In [7]:
# TRAINING BLOCK

# Array to store all final rewards
rewards_all_episodes = []

# Loop over all the episodes
for episode in range(num_episodes):
    print(f'Episode: {episode + 1}/{num_episodes}', end='\r')
    
#     Reset Environment before each episode begins
    state = env.reset()
    
#     done keeps track of whether or not the episode has ended
    done = False
    rewards_current_episode = 0

#     Run each step in an wpisode
    for step in range(max_steps_per_episode):
#         Get a random exploration threshold
        exploration_rate_threshold = random.uniform(0, 1)
        
#         Exploitation
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state, :])
#         Exploration
        else:
            action = env.action_space.sample()
        
#         Take the action
        new_state, reward, done, info = env.step(action)
        
#         Update the Q-Function
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
            learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))

        state = new_state
        rewards_current_episode += reward
        
#         If episode is over break out of this loop
        if done == True:
            break
    
#     Update Exploration Rate
    exploration_rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate * episode)
    rewards_all_episodes.append(rewards_current_episode)
    
# Display average per 1000 episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes), num_episodes / 1000)
count = 1000
print('*****Average rewards per thousand rewards*****')

for r in rewards_per_thousand_episodes:
    print(f'{count}: {sum(r/1000)}')
    count += 1000


*****Average rewards per thousand rewards*****
1000: 0.05300000000000004
2000: 0.20400000000000015
3000: 0.44000000000000034
4000: 0.6020000000000004
5000: 0.6580000000000005
6000: 0.7100000000000005
7000: 0.7060000000000005
8000: 0.7410000000000005
9000: 0.7490000000000006
10000: 0.7330000000000005


In [8]:
# Display the learned Q-Function
q_table

array([[0.53388668, 0.50793803, 0.50348681, 0.50213563],
       [0.37778246, 0.28058943, 0.23027297, 0.51798619],
       [0.4206554 , 0.39881638, 0.42140322, 0.48766767],
       [0.33335938, 0.3012729 , 0.33662116, 0.4763035 ],
       [0.55585494, 0.45640838, 0.35563161, 0.35649225],
       [0.        , 0.        , 0.        , 0.        ],
       [0.32759623, 0.19038875, 0.19602096, 0.13854938],
       [0.        , 0.        , 0.        , 0.        ],
       [0.47956199, 0.41481605, 0.43261246, 0.58338623],
       [0.33569352, 0.65063026, 0.35098708, 0.3002485 ],
       [0.65772026, 0.4171163 , 0.44687961, 0.32233445],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.49919183, 0.53281897, 0.76274494, 0.54849327],
       [0.74046701, 0.83259489, 0.72204179, 0.72513907],
       [0.        , 0.        , 0.        , 0.        ]])

In [9]:
# DEMO BLOCK
count = 0
for episode in range(3):
    state = env.reset()
    done = False
    print(f'*****Episode {episode + 1}*****')
    time.sleep(1)
    
    for step in range(max_steps_per_episode):
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)
        
        action = np.argmax(q_table[state, :])
        new_state, reward, done, info = env.step(action)
        
        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1:
                count += 1
                print('*****You Reached Your Goal*****')
            else:
                print('*****Fell Through A Hole')
            time.sleep(3)
            clear_output(wait=True)
            break
        
        state = new_state
    
env.close()
print(f'Won {count}/3 times.')

Won 2/3 times.
